# Import Dataset

In [1]:
import pandas as pd
import pandasql as ps

In [2]:
Incidents = pd.read_csv("dataset/Incidents.csv")
Shipments = pd.read_csv("dataset/Shipment.csv")
Summer_Medals = pd.read_csv("dataset/summer.csv")

# Dealing with Numerical

## Rounding Number

In [3]:
q = """
SELECT Cost, 
       ROUND(Cost, 0) AS RoundedCost
FROM Shipments
LIMIT 3
"""
ps.sqldf(q, locals())

,Cost,RoundedCost
0,11.2480,11.0
1,10.3976,10.0
2,13.2444,13.0


# Aggregation

In [4]:
q = """
SELECT AVG(DurationSeconds) AS Average,
        MIN(DurationSeconds) AS Minimum,
        MAX(DurationSeconds) AS Maximum
FROM Incidents
"""
ps.sqldf(q, locals())

,Average,Minimum,Maximum
0,5592.875783,0.08,10526400.0


In [5]:
q = """
SELECT Shape,
        AVG(DurationSeconds) AS Average,
        MIN(DurationSeconds) AS Minimum,
        MAX(DurationSeconds) AS Maximum
FROM Incidents
GROUP BY Shape
LIMIT 5
"""
ps.sqldf(q, locals())

,Shape,Average,Minimum,Maximum
0,None,18012.591549,1.00,2419200.0
1,changing,3191.674419,2.00,172800.0
2,chevron,1100.593750,2.00,21600.0
3,cigar,1736.405556,1.00,172800.0
4,circle,29999.860767,0.08,10526400.0


## Having Clause

In [6]:
q = """
-- Calculate the aggregations by Shape
SELECT Shape,
       AVG(DurationSeconds) AS Average, 
       MIN(DurationSeconds) AS Minimum, 
       MAX(DurationSeconds) AS Maximum
FROM Incidents
GROUP BY Shape
-- Return records where minimum of DurationSeconds is greater than 1
HAVING MIN(DurationSeconds) > 1
"""
ps.sqldf(q, locals())

,Shape,Average,Minimum,Maximum
0,changing,3191.674419,2.0,172800.0
1,chevron,1100.593750,2.0,21600.0
2,crescent,10.000000,10.0,10.0
3,cross,848.133333,2.0,7200.0
4,cylinder,795.241758,3.0,37800.0
5,egg,558.956140,1.5,7200.0
6,rectangle,969.613208,4.0,28800.0
7,teardrop,3501.685185,2.0,172800.0


## SUM

In [7]:
q = """
-- Write a query that returns an aggregation 
SELECT MixDesc, SUM(Quantity) AS Total
FROM Shipments
-- Group by the relevant column
GROUP BY MixDesc
LIMIT 5
"""
ps.sqldf(q, locals())

,MixDesc,Total
0,".40W/C 24"" SPD 5000PSI W/FIBER",26.5339
1,1 SACK ABC SLURRY,113.5673
2,1 SACK ABC SLURRY PUMPABLE,107.5198
3,1 SACK SAND SLURRY,115.7781
4,1 SACK SAND SLURRY PUMPABLE,108.1816


## Count

In [8]:
q = """
-- Count the number of rows by MixDesc
SELECT MixDesc, COUNT(MixDesc)
FROM Shipments
GROUP BY MixDesc
LIMIT 5
"""
ps.sqldf(q, locals())

,MixDesc,COUNT(MixDesc)
0,".40W/C 24"" SPD 5000PSI W/FIBER",1
1,1 SACK ABC SLURRY,5
2,1 SACK ABC SLURRY PUMPABLE,4
3,1 SACK SAND SLURRY,6
4,1 SACK SAND SLURRY PUMPABLE,5


# Missing Data

## Finding Not Null Value

In [9]:
q = """
-- Return the specified columns
SELECT IncidentDateTime, IncidentState
FROM Incidents
-- Exclude all the missing values from IncidentState  
WHERE IncidentState IS NOT NUll
LIMIT 5
"""
ps.sqldf(q, locals())

,IncidentDateTime,IncidentState
0,2005-10-31 18:00:00.000,ny
1,2005-10-31 18:30:00.000,nj
2,2005-10-31 19:00:00.000,md
3,2005-10-31 19:00:00.000,de
4,2005-10-31 19:00:00.000,az


## Imputing Missing Value

But what if you want to replace the missing values with another value instead of omitting them? You can do this using the ISNULL() function. 

ISNULL -> MS SERVER

IFNULL -> SQLITE

In [10]:
q = """
-- Check the IncidentState column for missing values and replace them with the City column
SELECT IncidentState, IFNULL(IncidentState, City) AS Location
FROM Incidents
-- Filter to only return missing values from IncidentState
WHERE IncidentState IS NULL
LIMIT 5
"""
ps.sqldf(q, locals(), )

,IncidentState,Location
0,None,australia
1,None,ivanka pri dunaji (slovakia)
2,None,tunisia (in-flight; over ocean)
3,None,dehradun (uttar pradesh) (india)
4,None,erode (india)


## Coalesce 

In [11]:
q = """
-- Replace missing values 
SELECT Country, IncidentState, City
FROM Incidents
WHERE Country IS NULL
LIMIT 5
"""
ps.sqldf(q, locals())

,Country,IncidentState,City
0,None,None,australia
1,None,None,ivanka pri dunaji (slovakia)
2,None,None,tunisia (in-flight; over ocean)
3,None,None,dehradun (uttar pradesh) (india)
4,None,None,erode (india)


Coalesce will replace a Null value with the first non null value from the other chosen tables

In [12]:
q = """
-- Replace missing values 
SELECT Country, COALESCE(Country, IncidentState, City) AS Location
FROM Incidents
WHERE Country IS NULL
LIMIT 5
"""
ps.sqldf(q, locals())

# This query create a new column called location where it takes input from the column country
# if country is missing, it will impute the value from either IncidentState or City

,Country,Location
0,None,australia
1,None,ivanka pri dunaji (slovakia)
2,None,tunisia (in-flight; over ocean)
3,None,dehradun (uttar pradesh) (india)
4,None,erode (india)


# CASE Statement

## One Condition

In [13]:
q = """
SELECT Country, 
       CASE WHEN Country = 'us'  THEN 'USA'
       ELSE 'International'
       END AS SourceCountry
FROM Incidents
LIMIT 3
"""
ps.sqldf(q, locals())

,Country,SourceCountry
0,us,USA
1,us,USA
2,us,USA


In [14]:
Incidents.head()

,IncidentDateTime,City,IncidentState,Country,Shape,DurationSeconds,Comments
0,2005-10-31 18:00:00.000,poughkeepsie,ny,us,light,37800.0,Several bright lights moving erratically for e...
1,2005-10-31 18:30:00.000,linwood,nj,us,light,5.0,VERY bright apparent meteor over Southern New ...
2,2005-10-31 19:00:00.000,clarksville,md,us,other,5.0,White ball shaped bright object whizzing acros...
3,2005-10-31 19:00:00.000,newark,de,us,light,45.0,Very fast&#44 brillant bluish/white light trav...
4,2005-10-31 19:00:00.000,scottsdale,az,us,triangle,600.0,Gilbert


## AND OR Conditition

In [15]:
q = """
SELECT IncidentState,
        CASE WHEN IncidentState = 'ny' OR IncidentState = 'md' THEN '-'
            ELSE '+'
            END AS Sign
FROM Incidents
LIMIT 5
"""
ps.sqldf(q, locals())

,IncidentState,Sign
0,ny,-
1,nj,+
2,md,-
3,de,+
4,az,+


## Based on a value in a list

In [16]:
q = """
SELECT IncidentState,
        CASE WHEN IncidentState IN ('ny', 'md') THEN 'zzz'
            ELSE '+'
            END AS Sign
FROM Incidents
LIMIT 5
"""
ps.sqldf(q, locals())

,IncidentState,Sign
0,ny,zzz
1,nj,+
2,md,zzz
3,de,+
4,az,+


Binning 

In [17]:
q = """
-- Complete the syntax for cutting the duration into different cases
SELECT DurationSeconds, 
-- Start with the 2 TSQL keywords, and after the condition a TSQL word and a value
      CASE WHEN (DurationSeconds <= 120) THEN 1
-- The pattern repeats with the same keyword and after the condition the same word and next value          
       WHEN (DurationSeconds > 120 AND DurationSeconds <= 600) THEN 2
-- Use the same syntax here             
       WHEN (DurationSeconds > 601 AND DurationSeconds <= 1200) THEN 3
-- Use the same syntax here               
       WHEN (DurationSeconds > 1201 AND DurationSeconds <= 5000) THEN 4
-- Specify a value      
       ELSE 5
       END AS SecondGroup   
FROM Incidents
LIMIT 5
"""
ps.sqldf(q, locals())

,DurationSeconds,SecondGroup
0,37800.0,5
1,5.0,1
2,5.0,1
3,45.0,1
4,600.0,2


# Dates

Source: [documentation](https://www.sqlitetutorial.net/sqlite-date-functions/sqlite-date-function/)

## Adding to Date

In [18]:
q = """
SELECT OrderDate, DATE(ShipDate, '+5 day') AS DeliveryDate
FROM Shipments
LIMIT 5
"""
ps.sqldf(q, locals())

,OrderDate,DeliveryDate
0,2017-09-27T06:50:26Z,2017-10-03
1,2016-06-23T08:48:19Z,2016-06-29
2,2016-06-16T09:05:09Z,2016-06-21
3,2016-06-23T13:58:49Z,2016-06-29
4,2016-06-18T08:36:08Z,2016-06-24


# Window Function

## ROW NUMBER

In [19]:
q = """
SELECT
  *,
  -- Assign numbers to each row
  ROW_NUMBER() OVER() AS Row_N
FROM Summer_Medals
ORDER BY Row_N ASC
LIMIT 3
"""
ps.sqldf(q, locals())

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Row_N
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold,1
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver,2
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze,3


In [20]:
q = """

SELECT Year, ROW_NUMBER() OVER() AS row_num
FROM (
    SELECT DISTINCT Year
    FROM Summer_Medals
    ORDER BY Year ASC
    )
LIMIT 6
"""
ps.sqldf(q, locals())

,Year,row_num
0,1896,1
1,1900,2
2,1904,3
3,1908,4
4,1912,5
5,1920,6


In [21]:
q = """
WITH sub AS (SELECT DISTINCT Year
                FROM Summer_Medals
                ORDER BY Year ASC)

SELECT Year, 
        ROW_NUMBER() OVER() AS row_num
FROM sub
LIMIT 6
"""
ps.sqldf(q, locals())

,Year,row_num
0,1896,1
1,1900,2
2,1904,3
3,1908,4
4,1912,5
5,1920,6


## Order By

Sub clause of OVER

Order inside OVER takes effect before outside order

In [22]:
q = """
WITH sub AS (SELECT DISTINCT Year
                FROM Summer_Medals
                ORDER BY Year ASC)

SELECT Year, 
        ROW_NUMBER() OVER(ORDER BY Year DESC) AS row_num
FROM sub
ORDER BY Year
LIMIT 6
"""
ps.sqldf(q, locals())

,Year,row_num
0,1896,27
1,1900,26
2,1904,25
3,1908,24
4,1912,23
5,1920,22


In [23]:
q = """

WITH Athlete_Medals AS(
SELECT Athlete, COUNT(*) AS medals
FROM Summer_Medals
GROUP BY Athlete)


SELECT *,
        ROW_NUMBER() OVER(ORDER BY medals DESC) AS rank
FROM Athlete_Medals
LIMIT 5
"""
ps.sqldf(q, locals())

,Athlete,medals,rank
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,2
2,ANDRIANOV Nikolay,15,3
3,MANGIAROTTI Edoardo,13,4
4,ONO Takashi,13,5


## LAG

In [24]:
q = """
WITH Weightlifting_Gold AS(
SELECT Year,
        Country AS Champion
FROM Summer_Medals
WHERE
    Discipline = 'Weightlifting' AND
    Gender = 'Men' AND
    Event = '69KG' AND
    Medal = 'Gold')
    
SELECT Year, Champion,
    LAG(Champion) OVER(ORDER BY Year ASC) AS prev
FROM Weightlifting_Gold
"""
ps.sqldf(q, locals())

,Year,Champion,prev
0,2000,BUL,None
1,2004,CHN,BUL
2,2008,CHN,CHN
3,2012,CHN,CHN


## Partition By

In [25]:
q = """
WITH Tennis_Gold AS (
  SELECT DISTINCT
    Gender, Year, Country
  FROM Summer_Medals
  WHERE
    Year >= 2000 AND
    Event = 'Javelin Throw' AND
    Medal = 'Gold')

SELECT Gender, Year, Country AS Champion, LAG(Country) OVER(PARTITION BY Gender) AS Last_Champion
FROM Tennis_Gold
"""
ps.sqldf(q, locals())

,Gender,Year,Champion,Last_Champion
0,Men,2000,CZE,None
1,Men,2004,NOR,CZE
2,Men,2008,NOR,NOR
3,Men,2012,TTO,NOR
4,Women,2000,NOR,None
5,Women,2004,CUB,NOR
6,Women,2008,CZE,CUB
7,Women,2012,CZE,CZE


In [26]:
q = """
WITH swim AS(
SELECT *
FROM Summer_Medals
WHERE Discipline = "Swimming" AND 
        Event = "100M Freestyle" AND 
        Medal = "Gold" AND 
        Year > 2000
ORDER BY Year ASC
)

SELECT Year, 
        Country, 
        Gender, 
        LAG(Country) OVER(PARTITION BY Gender) AS Previous
FROM swim
"""
ps.sqldf(q, locals())

,Year,Country,Gender,Previous
0,2004,NED,Men,None
1,2008,FRA,Men,NED
2,2012,USA,Men,FRA
3,2004,AUS,Women,None
4,2008,GER,Women,AUS
5,2012,NED,Women,GER


Patition by multiple columns:

In [27]:
q = """
WITH Athletics_Gold AS (
  SELECT DISTINCT
    Gender, Year, Event, Country
  FROM Summer_Medals
  WHERE
    Year >= 2000 AND
    Discipline = 'Athletics' AND
    Event IN ('100M', '10000M') AND
    Medal = 'Gold')
    
SELECT Gender, 
        Year, 
        Event, 
        Country AS Champion, 
        LAG(Country) OVER(PARTITION BY Gender, Event) AS Previous_Champion
FROM Athletics_Gold
"""
ps.sqldf(q, locals())

,Gender,Year,Event,Champion,Previous_Champion
0,Men,2000,10000M,ETH,None
1,Men,2004,10000M,ETH,ETH
2,Men,2008,10000M,ETH,ETH
3,Men,2012,10000M,GBR,ETH
4,Men,2000,100M,USA,None
5,Men,2004,100M,USA,USA
6,Men,2008,100M,JAM,USA
7,Men,2012,100M,JAM,JAM
8,Women,2000,10000M,ETH,None
9,Women,2004,10000M,CHN,ETH


## Fetching Function

* By default, a window starts at the beginning of the table or partition and ends at the current row. 

* RANGE BETWEEN Clause extends the window to the end of the table or partition.

### LEAD

In [28]:
q = """
WITH Discus_Medalists AS (
  SELECT DISTINCT
    Year,
    Athlete
  FROM Summer_Medals
  WHERE Medal = 'Gold'
    AND Event = 'Discus Throw'
    AND Gender = 'Women'
    AND Year >= 2000)
    
SELECT Year,
        Athlete, 
        LEAD(Athlete, 3) OVER() AS Ahead
FROM Discus_Medalists
ORDER BY Year
"""
ps.sqldf(q, locals())

,Year,Athlete,Ahead
0,2000,ZVEREVA Ellina,PERKOVIC Sandra
1,2004,SADOVA Natalya,None
2,2008,BROWN TRAFTON Stephanie,None
3,2012,PERKOVIC Sandra,None


### First Value

With absolute fetching functions like FIRST_VALUE, you can fetch a value at an absolute position in the table, like its beginning or end.

In [29]:
q = """
WITH All_Male_Medalists AS (
  SELECT DISTINCT
    Athlete
  FROM Summer_Medals
  WHERE Medal = 'Gold'
    AND Gender = 'Men')
    
SELECT Athlete, 
        FIRST_VALUE(Athlete) OVER(ORDER BY Athlete ASC) AS First_Athlete
FROM All_Male_Medalists
LIMIT 5
"""
ps.sqldf(q, locals())

,Athlete,First_Athlete
0,AABYE Edgar,AABYE Edgar
1,AALTONEN Paavo Johannes,AABYE Edgar
2,AAS Thomas Valentin,AABYE Edgar
3,ABALMASAU Aliaksei,AABYE Edgar
4,ABALO Luc,AABYE Edgar


### Last Value

In [30]:
Summer_Medals.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver


In [31]:
q = """
SELECT Year, 
        City, 
        FIRST_VALUE(City) OVER(ORDER BY Year) AS First_Value,
        LAST_VALUE(City) OVER (ORDER BY Year ASC
                                RANGE BETWEEN 
                                UNBOUNDED PRECEDING AND
                                UNBOUNDED FOLLOWING) AS Last_Value
FROM Summer_Medals
GROUP BY City
ORDER BY Year
"""
ps.sqldf(q, locals())

,Year,City,First_Value,Last_Value
0,1896,Athens,Athens,Beijing
1,1900,Paris,Athens,Beijing
2,1904,St Louis,Athens,Beijing
3,1908,London,Athens,Beijing
4,1912,Stockholm,Athens,Beijing
5,1920,Antwerp,Athens,Beijing
6,1928,Amsterdam,Athens,Beijing
7,1932,Los Angeles,Athens,Beijing
8,1936,Berlin,Athens,Beijing
9,1952,Helsinki,Athens,Beijing


## RANKING

ROW_NUMBER() = Based on a selected order

RANK = Give the same number if value the same, skip the next number

DENSE RANK = Doesn't skip rank

In [32]:
q = """
WITH Athlete_Medals AS (
  SELECT
    Athlete,
    COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete)
  
SELECT Athlete, 
        Medals,
        RANK() OVER(ORDER BY Medals DESC) AS Rank_N,
        DENSE_RANK() OVER(ORDER BY Medals DESC) AS Dense_Rank
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 10
"""
ps.sqldf(q, locals())

,Athlete,Medals,Rank_N,Dense_Rank
0,PHELPS Michael,22,1,1
1,LATYNINA Larisa,18,2,2
2,ANDRIANOV Nikolay,15,3,3
3,MANGIAROTTI Edoardo,13,4,4
4,ONO Takashi,13,4,4
5,SHAKHLIN Boris,13,4,4
6,COUGHLIN Natalie,12,7,5
7,FISCHER Birgit,12,7,5
8,KATO Sawao,12,7,5
9,NEMOV Alexei,12,7,5


In [33]:
#Rank each country's athletes by the count of medals they've earned -- the higher the count, 
#the higher the rank -- without skipping numbers in case of identical values.

q = """
WITH Athlete_Medals AS (
  SELECT
    Country, Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('JPN', 'KOR')
    AND Year >= 2000
  GROUP BY Country, Athlete
  HAVING COUNT(*) > 1)

SELECT Country, 
        Athlete, 
        Medals,
        RANK() OVER(PARTITION BY Country ORDER BY Medals DESC) AS Rank_N,
        DENSE_RANK() OVER(PARTITION BY Country ORDER BY Medals DESC) AS Dense_Rank
        
FROM Athlete_Medals
ORDER BY Country, Medals DESC
"""
ps.sqldf(q, locals())

,Country,Athlete,Medals,Rank_N,Dense_Rank
0,JPN,KITAJIMA Kosuke,7,1,1
1,JPN,UCHIMURA Kohei,5,2,2
2,JPN,TACHIBANA Miya,4,3,3
3,JPN,TAKEDA Miho,4,3,3
4,JPN,ICHO Kaori,3,5,4
...,...,...,...,...,...
69,KOR,OH Yong Ran,2,7,4
70,KOR,PARK Jinman,2,7,4
71,KOR,PARK Kyung-Mo,2,7,4
72,KOR,YOO Yong-Sung,2,7,4


## PAGING

### NTILE

There are exactly 666 unique events in the Summer Medals Olympics dataset. If you want to chunk them up to analyze them piece by piece, you'll need to split the events into groups of approximately equal size.

In [34]:
# Split the distinct events into exactly 111 groups, ordered by event in alphabetical order.
# Split the athletes into top, middle, and bottom thirds based on their count of medals.
q = """
WITH Events AS (
  SELECT DISTINCT Event
  FROM Summer_Medals)
  
SELECT Event,
        NTILE(111) OVER(ORDER BY Event ASC) AS Ntile
FROM Events
ORDER BY Event
"""
ps.sqldf(q, locals())

,Event,Ntile
0,+ 100KG,1
1,+ 100KG (Heavyweight),1
2,+ 100KG (Super Heavyweight),1
3,+ 105KG,1
4,+ 108KG Total (Super Heavyweight),1
...,...,...
661,York Round (100Y - 80Y - 60Y),111
662,Épée Amateurs And Masters,111
663,Épée Individual,111
664,Épée Masters,111


In [35]:
#PART 1
# Split the athletes into top, middle, and bottom thirds based on their count of medals.
q = """
WITH Athlete_Medals AS (
  SELECT Athlete, 
          COUNT(*) AS Medals
          
  FROM Summer_Medals
  GROUP BY Athlete
  HAVING COUNT(*) > 1)

SELECT Athlete, 
        Medals,
        NTILE(3) OVER(ORDER BY Medals DESC) AS Third
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 5
"""
ps.sqldf(q, locals())

,Athlete,Medals,Third
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,1
2,ANDRIANOV Nikolay,15,1
3,MANGIAROTTI Edoardo,13,1
4,ONO Takashi,13,1


In [36]:
#PART 2
# Return the average of each third.
q = """
WITH Athlete_Medals AS (
  SELECT Athlete, 
          COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete
  HAVING COUNT(*) > 1),

Third AS (
    SELECT Athlete, 
    Medals,
    NTILE(3) OVER(ORDER BY Medals DESC) AS Third
    FROM Athlete_Medals)


SELECT AVG(Medals) AS avg_medals
FROM Third
GROUP BY Third
"""
ps.sqldf(q, locals())

,avg_medals
0,3.786446
1,2.000000
2,2.000000


## Aggregate Window Functions

### Sum (Running Total/Cumsum)

In [37]:
# Return the athletes, the number of medals they earned, and the medals running total, 
# ordered by the athletes' names in alphabetical order.
q = """
WITH Athlete_Medals AS (
  SELECT
    Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'USA' AND Medal = 'Gold'
    AND Year >= 2000
  GROUP BY Athlete)

SELECT Athlete, 
        Medals,
        SUM(Medals) OVER(ORDER BY Athlete ASC) AS Max_Medals
FROM Athlete_Medals
ORDER BY Athlete ASC
LIMIT 5
"""
ps.sqldf(q, locals())

,Athlete,Medals,Max_Medals
0,ABDUR-RAHIM Shareef,1,1
1,ABERNATHY Brent,1,2
2,ADRIAN Nathan,3,5
3,AHRENS Chris,1,6
4,AINSWORTH Kurt,1,7


### MAX

In [38]:
# Return the year, country, medals, and the maximum medals earned so far for each country, 
# ordered by year in ascending order.
q = """
WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('CHN', 'KOR', 'JPN')
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year, Country)

SELECT Year, Country, Medals,
        MAX(Medals) OVER(PARTITION BY Country
                            ORDER BY Year ASC) AS Max_Medals
FROM Country_Medals
"""
ps.sqldf(q, locals())

,Year,Country,Medals,Max_Medals
0,2000,CHN,39,39
1,2004,CHN,52,52
2,2008,CHN,74,74
3,2012,CHN,56,74
4,2000,JPN,5,5
5,2004,JPN,21,21
6,2008,JPN,23,23
7,2012,JPN,7,23
8,2000,KOR,12,12
9,2004,KOR,14,14


### MIN

In [39]:
# Return the year, medals earned, and minimum medals earned so far.
q = """
WITH France_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'FRA'
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year)
  
SELECT Year, Medals,
        MIN(Medals) OVER(ORDER BY Year ASC) AS Min_Medals
FROM France_Medals
"""
ps.sqldf(q, locals())

,Year,Medals,Min_Medals
0,2000,22,22
1,2004,21,21
2,2008,25,21
3,2012,30,21


## FRAMEs

FRAME example: RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING

Syntax:

ROWS BETWEEN vs. RANGE BETWEEN

ROWS BETWEEN treats duplicates value as as multiple rows
RANGE BETWEEN treats duplicates in OVER's ORDER BY subcause as a single entity


### UNBOUNDED

In [40]:
q = """
SELECT Year, 
        City, 
        FIRST_VALUE(City) OVER(ORDER BY Year) AS First_Value,
        LAST_VALUE(City) OVER (ORDER BY Year ASC
                                RANGE BETWEEN 
                                UNBOUNDED PRECEDING AND
                                UNBOUNDED FOLLOWING) AS Last_Value
FROM Summer_Medals
GROUP BY City
ORDER BY Year
LIMIT 5
"""
ps.sqldf(q, locals())

,Year,City,First_Value,Last_Value
0,1896,Athens,Athens,Beijing
1,1900,Paris,Athens,Beijing
2,1904,St Louis,Athens,Beijing
3,1908,London,Athens,Beijing
4,1912,Stockholm,Athens,Beijing


The query above uses frame. But, without frame Last_Value(City) will just return City. Thats because, by default, a frame starts at the beginning of the table or partition till the current row. So last value with default frame will be equal to the current row:

In [41]:
q = """
SELECT Year, 
        City, 
        FIRST_VALUE(City) OVER(ORDER BY Year) AS First_Value,
        LAST_VALUE(City) OVER (ORDER BY Year ASC) AS Last_Value
FROM Summer_Medals
GROUP BY City
ORDER BY Year
LIMIT 5
"""
ps.sqldf(q, locals())

,Year,City,First_Value,Last_Value
0,1896,Athens,Athens,Athens
1,1900,Paris,Athens,Paris
2,1904,St Louis,Athens,St Louis
3,1908,London,Athens,London
4,1912,Stockholm,Athens,Stockholm


### ROWS BETWEEN

In [42]:
# Return the year, medals earned, and the maximum medals earned, 
# comparing only the current year and the next year.

q = """
WITH Scandinavian_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('DEN', 'NOR', 'FIN', 'SWE', 'ISL')
    AND Medal = 'Gold'
  GROUP BY Year)

SELECT Year, 
        Medals, 
        MAX(Medals) OVER(ORDER BY Year ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) AS Max_Medals

FROM Scandinavian_Medals
LIMIT 10
"""
ps.sqldf(q, locals())

,Year,Medals,Max_Medals
0,1896,1,1
1,1900,1,77
2,1908,77,141
3,1912,141,159
4,1920,159,159
5,1924,48,48
6,1928,24,24
7,1932,17,17
8,1936,15,54
9,1948,54,54


In [43]:
# Return the athletes, medals earned, and the maximum medals earned, comparing only the last two and 
# current athletes, ordering by athletes' names in alphabetical order.

q = """
WITH Chinese_Medals AS (
  SELECT
    Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'CHN' AND Medal = 'Gold'
    AND Year >= 2000
  GROUP BY Athlete)

SELECT Athlete, Medals,
        MAX(Medals) OVER(ORDER BY Athlete ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Max_Medals
FROM Chinese_Medals
LIMIT 10
"""
ps.sqldf(q, locals())

,Athlete,Medals,Max_Medals
0,CAI Yalin,1,1
1,CAI Yun,1,1
2,CAO Lei,1,1
3,CAO Yuan,1,1
4,CHEN Ding,1,1
5,CHEN Jing,1,1
6,CHEN Qi,1,1
7,CHEN Ruolin,4,4
8,CHEN Xiaomin,1,4
9,CHEN Xiexia,1,4


In [44]:
#Calculate the 3-year moving average of medals earned.
q = """
WITH Russian_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'RUS'
    AND Medal = 'Gold'
    AND Year >= 1980
  GROUP BY Year)

SELECT Year, 
        Medals, 
        AVG(Medals) OVER(ORDER BY YEAR ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Medals_MA
FROM Russian_Medals
"""
ps.sqldf(q, locals())

,Year,Medals,Medals_MA
0,1996,36,36.000000
1,2000,66,51.000000
2,2004,47,49.666667
3,2008,43,52.000000
4,2012,47,45.666667
